Cristian Villazhannay 
ME449 
6 Nov 2023

Scroll down for the report.

In [3]:
#import necessary packages
import math
import numpy as np
import modern_robotics as mr 

In [4]:
def Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength):
    '''
    Puppet is an interactive animation tool that allows the user to grab the link 
    of a virtual mechanism and apply forces to it, to make it move like a marionette.

    Parameters
    ----------
    thetalist   : np.array
        an (n,) shaped vector of initial joint angles (rad)

    dthetalist  : np.array
        an (n,) shaped vector of initial joint rates (rad/s)

    g           : np.array
        a (3,) shaped vector in the {s} frame (m/s^2)

    Mlist       : np.array
        the configurations of the link frames relative to each other at the home configuation. 
        for N-links the matrix will be (N,16) with row i corresponding to M_{i,i+1}. 
        Use the .flatten() method to create a single row. 

    Slist       : np.array
        the screw axes S_i in the space frame when the robot is in its home configuration. 
        for N-links, the matrix will be (N,6) with row i corresponsing to S_i

    Glist       : np.array
        the spatial inertia matrices G_i of the links (kg and kg*m^2)

    t           : float
        the total simulation time (s)

    dt          : float
        the simulation timestep (s)

    damping     : int
        a scalar indicating the viscous damping at each joint. (Nms/rad)

    stiffness   : int
        a scalar indicating the stiffness of the spring string. 

    springPos   : np.array
        a (3,) vector indicating the location of the end of the spring not attached to the robot,
        expressed in the {s} frame (m)

    restLength  : int
        a scalar indicating the length of the spring when it is at rest. (m)

    Outputs
    ----------
    thetamat  : np.array
        An N x n matrix where row i is the set of joint values after simulation step i - 1.
    
    dthetamat : np.array
        An N x n matrix where row i is the set of joint rates after simulation step i - 1. 
    '''
    #implement type verification.
    #set joint size throughout.
    n = thetalist.size
    
    #find out how many steps we have to animate. Round up to ensure we have room. 
    N = math.ceil(t / dt) 

    #create empty arrays for allocation
    thetamat = np.empty((N,n))
    dthetamat = np.empty((N,n))

    #initialize our current theta and dtheta. 
    #will also help standardize our input in case it wasn't already an array.
    currTheta = np.copy(thetalist)
    currDTheta = np.copy(dthetalist)

    #find the home configuration of end effector matrix. This is equivalent to T_sbhome
    endHome = Mlist[0,:,:] 
    for ii in range(Mlist.shape[0])[1:]:
        endHome = endHome @ Mlist[ii,:,:]
        

    for i in range(0, N):
        #index our current theta into the matrix. 
        thetamat[i, :] = currTheta
        dthetamat[i, :] = currDTheta

        #changes based on where the angles are. Calculate the T_sb based on the current angles.
        T_sb = mr.FKinSpace(endHome, Slist, currTheta)

        #at the beginning, there will be no torques. The arm will go limp. 
        #units of (N*m)
        tau = np.zeros((6,))

        #When damping is accounted for in the equation. 
        tau = tau - damping * currDTheta

        #convert spring position to the b-frame. extract only the data we need.
        springBPos = (T_sb.T @ np.append(springPos, 1))[0:3]

        #since the spring position is now in the B frame it will subtract 0 every time.
        dist = np.linalg.norm(springBPos)
        forceMag = stiffness * (dist - restLength)
        unitVec = springBPos / dist

        #Higher damping forces cause greater oscillation as the high torques cause the robot arm to overshoot the position. 

        #is the distance between the springPos > restLength? 
        #There will be no rotational forces. Only a linear force in the unit direction the spring.
        ftip = np.array([0, 0, 0, unitVec[0], unitVec[1], unitVec[2]]) * forceMag * -1
            
        #eventually, forces at the tip will be dicated by the spring acting on the body-effector frame.
        #will have to be calculated each frame as they will change with each movement.
        # ftip = np.array([0, 0, 0, 0, 0, 0])

        #acceleration list based on ForwardDynamics
        ddthetalist = mr.ForwardDynamics(currTheta, currDTheta, tau, g, ftip, Mlist, Glist, Slist)

        #get the next step from Euler Integration
        currTheta, currDTheta = mr.EulerStep(currTheta, currDTheta, ddthetalist, dt)

    # #change out values so that they are in the [-pi, pi] range
    # thetamat = np.arctan2(np.sin(thetamat), np.cos(thetamat))

    #save the thetamat in a csv for CoppeliaSim.
    np.savetxt('theta' + '.csv', thetamat, delimiter=',')
        
    #outputs
    return thetamat, dthetamat

In [15]:
#Parameters that do not change. 
M01 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.089159], [0, 0, 0, 1]]
M12 = [[0, 0, 1, 0.28], [0, 1, 0, 0.13585], [-1, 0, 0, 0], [0, 0, 0, 1]]
M23 = [[1, 0, 0, 0], [0, 1, 0, -0.1197], [0, 0, 1, 0.395], [0, 0, 0, 1]]
M34 = [[0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0.14225], [0, 0, 0, 1]]
M45 = [[1, 0, 0, 0], [0, 1, 0, 0.093], [0, 0, 1, 0], [0, 0, 0, 1]]
M56 = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.09465], [0, 0, 0, 1]]
M67 = [[1, 0, 0, 0], [0, 0, 1, 0.0823], [0, -1, 0, 0], [0, 0, 0, 1]]
G1 = np.diag([0.010267495893, 0.010267495893,  0.00666, 3.7, 3.7, 3.7])
G2 = np.diag([0.22689067591, 0.22689067591, 0.0151074, 8.393, 8.393, 8.393])
G3 = np.diag([0.049443313556, 0.049443313556, 0.004095, 2.275, 2.275, 2.275])
G4 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G5 = np.diag([0.111172755531, 0.111172755531, 0.21942, 1.219, 1.219, 1.219])
G6 = np.diag([0.0171364731454, 0.0171364731454, 0.033822, 0.1879, 0.1879, 0.1879])
Glist = [G1, G2, G3, G4, G5, G6]
Mlist = [M01, M12, M23, M34, M45, M56, M67] 
Slist = [[0,         0,         0,         0,        0,        0],
         [0,         1,         1,         1,        0,        1],
         [1,         0,         0,         0,       -1,        0],
         [0, -0.089159, -0.089159, -0.089159, -0.10915, 0.005491],
         [0,         0,         0,         0,  0.81725,        0],
         [0,         0,     0.425,   0.81725,        0,  0.81725]]

#Standardize our inputs to be arrays.
thetalist = np.array([0, 0, 0, 0, 0, 0])
dthetalist = np.array([0, 0, 0, 0, 0, 0])
# g = np.array([0, 0, -9.81])
g = np.array([0, 0, 0])
Mlist = np.asanyarray(Mlist)
Glist = np.asanyarray(Glist)
Slist = np.asanyarray(Slist)
t = 10
dt = 0.01
damping = 0
stiffness = 50
springPos = np.array([1, 1, 1])
restLength = 0

thetamat, dthetamat = Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength)

### Introduction
This submission is created in a Jupyter Notebook through the Anaconda environment.

In this submission, we begin by simply import libraries, declare the `Puppet` function, and initialize the parameters that will not be changing like the robot angles and screw axes. These are outlined in the first 3 cells, which must be run before any of the function calls.

The rest of the code is organized and labelled in their respective section. 

To run the rest of the code, simply press `Shift + Enter` to run a cell or click the `Run` button. Please note that the function will only generate a .CSV file called theta.csv. When running multiple code blocks with the same function call, this .csv file _will_ be overwritten. To prevent this, rename the .csv file after creation. 

### Part 1: Simulating a Falling Robot
a. Choose a value of dt where the energy of the system appears nearly constant.

In [217]:
#Parameters to create part 1a. 
thetalist = np.array([0, 0, 0, 0, 0, 0])
dthetalist = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])
Mlist = np.asanyarray(Mlist)
Glist = np.asanyarray(Glist)
Slist = np.asanyarray(Slist)
t = 5
dt = 0.001   #The timestep is very fine, enabling a great resolution even with 
             #Euler Integration
damping = 0
stiffness = 0
springPos = np.array([1, 1, 1])
restLength = 0

thetamat, dthetamat = Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength)

b. Choose a value of dt where the energy of the system does not appear constant

In [ ]:
#Parameters to create part 1b. 
thetalist = np.array([0, 0, 0, 0, 0, 0])
dthetalist = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])
Mlist = np.asanyarray(Mlist)
Glist = np.asanyarray(Glist)
Slist = np.asanyarray(Slist)
t = 5
dt = 0.01       #The timestep is quite coarse, leading to accumulating errors
                #with Euler Integration. 
damping = 0
stiffness = 0
springPos = np.array([1, 1, 1])
restLength = 0

thetamat, dthetamat = Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength)

c. Explain how you would calculate the total energy of the robot at each timestep, if you wanted to plot the total energy to confirm that your simulation approximately preserves it. 

In order to calculate the total energy of the robot at the different timesteps, I would calculate the potential energy and kinetic energy of the robot at each point. If energy is conserved throughout, then the potential energy and the kinetic energy should always sum up to the same value. This would likely be accomplished through an implementation of Lagrangian Dynamics. 

### Part 2: Adding Damping
a. When you choose damping to be positive, the robot loses energy as it swings. 

In [208]:
#Parameters to create part 2a. 
thetalist = np.array([0, 0, 0, 0, 0, 0])
dthetalist = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])
Mlist = np.asanyarray(Mlist)
Glist = np.asanyarray(Glist)
Slist = np.asanyarray(Slist)
t = 5
dt = 0.01
damping = 3
stiffness = 0
springPos = np.array([1, 1, 1])
restLength = 0

thetamat, dthetamat = Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength)

b. When you choose damping to be negative, the robot gains energy as it swings. 


In [209]:
#Parameters to create part 2b. 
thetalist = np.array([0, 0, 0, 0, 0, 0])
dthetalist = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])
Mlist = np.asanyarray(Mlist)
Glist = np.asanyarray(Glist)
Slist = np.asanyarray(Slist)
t = 5
dt = 0.01
damping = -0.01
stiffness = 0
springPos = np.array([1, 1, 1])
restLength = 0

thetamat, dthetamat = Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength)

c. Do you see any strange behavior in the simulation if you choose the damping constant to be a large positive value? Can you explain it? How would this behavior change if you chose shorter simulation timesteps?

The strange behavior that I see in the simulation if I choose the damping constant to be a large positive value - in my simulation this is a value of 4 and above - is that the theta values quickly grow towards infinity, leading to overflow errors or the generation of _NaN_ values in Python. This leads to a CoppeliaSim .csv that is unable to run since CoppeliaSim cannot translate these numbers approaching infinity. I believe that the higher positive damping force leads the damping torques to create overshoot in the theta angle position. This overshoot then causes the following damping force to overcorrect, leading to large oscillatory torques that cause the robot arm to swing at larger angles until they approach infinity. 

When the timestep is shortened, the function is capable of handling damping values of 4 and above. The reason why is likely because the overshoot error is minimized when the time horizon over which it is calculated (i.e, dt = 0.001) is smaller. This leads to less drastic overshoot and subsequently smaller overcorrections. 

### Part 3: Adding a Spring
a. Capture a video for a stiffness that makes the robot oscillate a couple of times and record the stiffness value. Considering the system's total energy, does the motion of the robot make sense? What do you expect to happen to the total energy over time? Describe any strange behavior you see if you choose the spring constant to be too large. If you don't see any strange behavior, explain why. 

I believe that the total motion of the robot does make sense. I expect to see the total energy of the system to remain constant over time. Since there is no gravity, and no other external forces (i.e, damping) then I believe that the system will conserve its energy and oscillate forever. If I set the spring constant to be too large, the strange behavior observed is the robot begins to act erratically. The links begin to be pulled in seemingly random directions and the end effector begins to gain energy. 




In [12]:
#Standardize our inputs to be arrays.
thetalist = np.array([0, 0, 0, 0, 0, 0])
dthetalist = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, 0])
Mlist = np.asanyarray(Mlist)
Glist = np.asanyarray(Glist)
Slist = np.asanyarray(Slist)
t = 10
dt = 0.01
damping = 0
stiffness = 6
springPos = np.array([1, 1, 1])
restLength = 0

thetamat, dthetamat = Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength)

b. Now add a positive damping to the simulation that makes the arm nearly come to rest by the end of the video. 

In [216]:
#Standardize our inputs to be arrays.
thetalist = np.array([0, 0, 0, 0, 0, 0])
dthetalist = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, 0])
Mlist = np.asanyarray(Mlist)
Glist = np.asanyarray(Glist)
Slist = np.asanyarray(Slist)
t = 10
dt = 0.01
damping = 1
stiffness = 6
springPos = np.array([1, 1, 1])
restLength = 0

thetamat, dthetamat = Puppet(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, springPos, restLength)